In [1]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

import analysis_helpers as h
reload(h)

## get standardized object list
categories = ['bird','car','chair','dog']
obj_list = []
for cat in categories:
    for i,j in h.objcat.iteritems():
        if j==cat:
            obj_list.append(i)        

<module 'analysis_helpers' from 'analysis_helpers.pyc'>

In [2]:
# directory & file hierarchy
iterationName = 'pilot2'
exp_path = './'
analysis_dir = os.getcwd()
data_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','data',exp_path))
exp_dir = './'
sketch_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','analysis',exp_path,'sketches','pilot2'))

In [5]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['sketchpad_basic_recog']

stimdb = conn['stimuli']
stimcoll = stimdb['sketchpad_basic_pilot2_sketches']

In [84]:
## How many sketches have been retrieved at least once? equivalent to: coll.find({'numGames':{'$exists':1}}).count()
x = stimcoll.find({'numGames':{'$gte':0}}).count()
y = coll.count()
print '{} sketches in the stimuli db that have been retrieved at least once'.format(x)
print '{} records in the recognition experiment database'.format(y)

2849 sketches in the stimuli db that have been retrieved at least once
7720 records in the recognition experiment database


### preprocess recognition task data

In [119]:
## retrieve records from db
## notes: 
## pilot0 = no feedback onscreen
## pilot1 = bonus point counter onscreen

a = coll.find({'iterationName': {'$in': ['pilot0', 'pilot1']}}).sort('gameID')

## make lists from db
gameID = []
target = []
choice = []
correct = []
correct_class = []
rt = []
fname = []

d1 = []
d2 = []
d3 = []
target_category = []
chosen_category = []
condition = []
drawDuration = []
original_gameID = []
viewer_correct = []
viewer_choice = []
viewer_RT = []
mean_intensity = []
num_strokes = []

bad_sessions = ['1571-00d11ddf-96e7-4aae-ba09-1a338b328c0e','9770-2f360e9a-7a07-4026-9c36-18b558c1da21']

counter = 0
for rec in a:
    if rec['gameID'] not in bad_sessions:
        try:
            if counter%500==0:
                print '{} out of {} records analyzed.'.format(counter,a.count())
            if rec['target'] is not None:
                gameID.append(rec['gameID'])
                target.append(rec['target'])
                choice.append(rec['choice'])
                correct.append(rec['correct'])
                rt.append(rec['rt'])
                f = rec['sketch'].split('/')[-1]
                fname.append(f)
                chosen_category.append(h.objcat[rec['choice']])

                ## match up with corresponding record in stimuli collection
                b = stimcoll.find({'fname_no_target':f})[0]
                assert stimcoll.find({'fname_no_target':f}).count()==1
                d1.append(b['Distractor1'])
                d2.append(b['Distractor2'])
                d3.append(b['Distractor2'])
                target_category.append(b['category'])
                correct_class.append(h.objcat[rec['choice']]==b['category'])
                condition.append(b['condition'])
                drawDuration.append(b['drawDuration'])
                original_gameID.append(b['gameID'])
                viewer_correct.append(b['outcome'])
                viewer_choice.append(b['response'])
                viewer_RT.append(b['viewerRT'])
                mean_intensity.append(b['mean_intensity'])  
                num_strokes.append(b['numStrokes'])    
                counter += 1
        except:
            print 'Something went wrong with {} {}'.format(rec['gameID'],rec['trialNum'])
            pass

0 out of 7709 records analyzed.
500 out of 7709 records analyzed.
1000 out of 7709 records analyzed.
1500 out of 7709 records analyzed.
2000 out of 7709 records analyzed.
2500 out of 7709 records analyzed.
3000 out of 7709 records analyzed.
3500 out of 7709 records analyzed.
4000 out of 7709 records analyzed.
4500 out of 7709 records analyzed.
5000 out of 7709 records analyzed.
5500 out of 7709 records analyzed.
Something went wrong with 7699-7141d5a7-5a42-42e7-85dc-ae36f351bfda 47
6000 out of 7709 records analyzed.
6500 out of 7709 records analyzed.
7000 out of 7709 records analyzed.
7500 out of 7709 records analyzed.


In [191]:
## organize data into dataframe
X = pd.DataFrame([gameID,target,choice,correct,rt,fname,d1,d2,d3,target_category,chosen_category,condition,drawDuration, \
                 original_gameID,viewer_correct,viewer_choice,viewer_RT,mean_intensity,num_strokes,correct_class])
X = X.transpose()
X.columns = ['gameID','target','choice','correct','rt','fname','d1','d2','d3','target_category','chosen_category','condition','drawDuration', \
            'original_gameID','viewer_correct','viewer_choice','viewer_RT','mean_intensity','num_strokes','correct_class']
print '{} annotations saved.'.format(X.shape[0])

In [186]:
## remove NaN rows from data matrix (no target recorded)
X = X[X['target'].isin(obj_list)]

## filter out responses that took too long, or too short
too_fast = 1000
too_slow = 20000
X = X[(X['rt']>=too_fast) & (X['rt']<=too_slow)]

print '{} annotations retained.'.format(X.shape[0])

7017 annotations retained.


In [192]:
## save out to CSV
X.to_csv('./sketchpad_basic_recog_group_data.csv')

#### basic performance measures 

In [161]:
if not os.path.exists('./plots'):
    os.makedirs('./plots')

In [194]:
## what is object-level accuracy broken out condition?
print X.groupby('condition')['correct'].apply(lambda x: np.mean(x))

## what is class-level accuracy?
print X.groupby('condition')['correct_class'].apply(lambda x: np.mean(x))


condition
closer     0.541994
further    0.374720
Name: correct, dtype: float64
condition
closer     0.980238
further    0.978188
Name: correct_class, dtype: float64


In [164]:
print X.groupby('condition')['rt'].apply(lambda x: np.median(x))

condition
closer     4052.0
further    4131.5
Name: rt, dtype: float64


In [125]:
# X.groupby(['gameID','condition'])['correct'].apply(lambda x: np.mean(x))
# X.groupby(['gameID'])['correct'].apply(lambda x: np.mean(x))

In [180]:
## plot accuracy by condition and game (individual differences)
#X.groupby(['gameID','condition'])['correct'].apply(lambda x: np.mean(x))

## subset by full games only
all_games = np.unique(X.gameID.values)
full_games = [i for i in all_games if np.sum(X['gameID']==i)>50]
_X = X[X['gameID'].isin(full_games)]

game_acc_close = _X[_X['condition']=='closer'].groupby('gameID')['correct'].apply(lambda x: np.mean(x))
game_acc_far = _X[_X['condition']=='further'].groupby('gameID')['correct'].apply(lambda x: np.mean(x))
fig = plt.figure(figsize=(6,6))
plt.scatter(game_acc_close,game_acc_far)
plt.xlim([0,1])
plt.ylim([0,1])
plt.plot([0,1],[0,1],linestyle='dashed')
plt.title('accuracy by condition and game')
plt.xlabel('close accuracy')
plt.ylabel('far accuracy')
plt.savefig('./plots/accuracy_by_condition_and_game.pdf')
plt.close(fig)

In [181]:
## overall accuracy by category
fig = plt.figure(figsize=(6,6))
sns.barplot(x='target_category',y='correct',data=X)
plt.xlabel('category')
plt.ylabel('accuracy')
plt.title('accuracy by category')
plt.ylim([0,1])
plt.savefig('./plots/accuracy_by_category.pdf')
plt.close(fig)

In [182]:
objs = np.unique(X['target'].values)
objs = [o for o in objs if o is not None]
obj_acc_close = X[X['condition']=='closer'].groupby('target')['correct'].apply(lambda x: np.mean(x))
obj_acc_far = X[X['condition']=='further'].groupby('target')['correct'].apply(lambda x: np.mean(x))
fig = plt.figure(figsize=(12,12))
plt.scatter(obj_acc_close,obj_acc_far)
for i, txt in enumerate(objs):
    plt.annotate(txt, (obj_acc_close[i],obj_acc_far[i]))
plt.xlim([-0.1,1])
plt.ylim([-0.1,1]) 
plt.plot([0,1],[0,1],linestyle='dashed')
plt.xlabel('close accuracy')
plt.ylabel('far accuracy')
plt.title('accuracy by condition and object')
plt.savefig('./plots/accuracy_by_condition_and_object.pdf')
plt.close(fig)

In [183]:
objs = np.unique(X['target'].values)
objs = [o for o in objs if o is not None]
obj_acc_close = X[X['condition']=='closer'].groupby('target')['rt'].apply(lambda x: np.mean(x))
obj_acc_far = X[X['condition']=='further'].groupby('target')['rt'].apply(lambda x: np.mean(x))
fig = plt.figure(figsize=(12,12))
plt.scatter(obj_acc_close,obj_acc_far)
for i, txt in enumerate(objs):
    plt.annotate(txt, (obj_acc_close[i],obj_acc_far[i]))
plt.xlim([0,20000])
plt.ylim([0,20000])
plt.plot([0,20000],[0,20000],linestyle='dashed')
plt.xlabel('close RT')
plt.ylabel('far RT')
plt.title('RT by condition and object')
plt.savefig('./plots/RT_by_condition_and_object.pdf')
plt.close(fig)

In [184]:
## subset by full games only
all_games = np.unique(X.gameID.values)
full_games = [i for i in all_games if np.sum(X['gameID']==i)>50]
_X = X[X['gameID'].isin(full_games)]

game_acc_close = _X[_X['condition']=='closer'].groupby('gameID')['rt'].apply(lambda x: np.median(x))
game_acc_far = _X[_X['condition']=='further'].groupby('gameID')['rt'].apply(lambda x: np.median(x))
fig = plt.figure(figsize=(6,6))
plt.scatter(game_acc_close,game_acc_far)
# plt.xlim([0,20000])
# plt.ylim([0,20000])
# plt.plot([0,20000],[0,20000],linestyle='dashed')
plt.title('RT by condition and game')
plt.xlabel('close RT')
plt.ylabel('far RT')
plt.savefig('./plots/RT_by_condition_and_game.pdf')
plt.close(fig)

In [185]:
## subset by full games only
all_games = np.unique(X.gameID.values)
full_games = [i for i in all_games if np.sum(X['gameID']==i)>50]
_X = X[X['gameID'].isin(full_games)]

acc = _X.groupby('gameID')['correct'].apply(lambda x: np.mean(x))
rt = _X.groupby('gameID')['rt'].apply(lambda x: np.mean(x))
fig = plt.figure(figsize=(6,6))
plt.scatter(acc,rt)
plt.xlabel('accuracy')
plt.ylabel('RT')
plt.title('RT vs. accuracy by game')
plt.savefig('./plots/RT_vs_accuracy_by_game.pdf')
plt.close(fig)

In [171]:
# Y = X.groupby(['target','condition'])['correct'].apply(lambda x: np.mean(x))

### make confusion matrix

In [172]:
import analysis_helpers as h
reload(h)

## get standardized object list
categories = ['bird','car','chair','dog']
obj_list = []
for cat in categories:
    for i,j in h.objcat.iteritems():
        if j==cat:
            obj_list.append(i)        

##### all sketches

In [173]:
## initialize confusion matrix
confusion = np.zeros((len(obj_list),len(obj_list)))

## generate confusion matrix by incrementing in each cell
for i,d in X.iterrows():
    targ_ind = obj_list.index(d['target'])
    choice_ind = obj_list.index(d['choice'])
    confusion[targ_ind,choice_ind] += 1
    
## normalized confusion matrix    
normed = np.zeros((len(obj_list),len(obj_list)))
for i in np.arange(len(confusion)):
    normed[i,:] = confusion[i,:]/np.sum(confusion[i,:])    
    
## plot confusion matrix
from matplotlib import cm
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(111)
cax = ax.matshow(normed,vmin=0,vmax=1,cmap=cm.viridis)
plt.xticks(range(len(normed)), obj_list, fontsize=12,rotation='vertical')
plt.yticks(range(len(normed)), obj_list, fontsize=12)
plt.colorbar(cax,shrink=0.8)
plt.tight_layout()
plt.savefig('./plots/confusion_matrix_all.pdf')
plt.close(fig)

##### divided by condition

In [174]:
conds = ['closer','further']

for cond in conds:
    ## initialize confusion matrix 
    confusion = np.zeros((len(obj_list),len(obj_list)))

    _X = X[X['condition']==cond]
    ## generate confusion matrix by incrementing in each cell
    for i,d in _X.iterrows():
        targ_ind = obj_list.index(d['target'])
        choice_ind = obj_list.index(d['choice'])
        confusion[targ_ind,choice_ind] += 1

    ## normalized confusion matrix    
    normed = np.zeros((len(obj_list),len(obj_list)))
    for i in np.arange(len(confusion)):
        normed[i,:] = confusion[i,:]/np.sum(confusion[i,:])    

    ## plot confusion matrix
    from matplotlib import cm
    fig = plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    cax = ax.matshow(normed,vmin=0,vmax=1,cmap=cm.viridis)
    plt.xticks(range(len(normed)), obj_list, fontsize=12,rotation='vertical')
    plt.yticks(range(len(normed)), obj_list, fontsize=12)
    plt.colorbar(cax,shrink=0.8)
    plt.tight_layout()
    plt.savefig('./plots/confusion_matrix_{}.pdf'.format(cond))
    plt.close(fig)

##### plot difference between close and far conditions

In [218]:
conds = ['closer','further']
normed = np.zeros((len(obj_list),len(obj_list),2))

for k,cond in enumerate(conds):
    ## initialize confusion matrix 
    confusion = np.zeros((len(obj_list),len(obj_list)))

    _X = X[X['condition']==cond]
    ## generate confusion matrix by incrementing in each cell
    for i,d in _X.iterrows():
        targ_ind = obj_list.index(d['target'])
        choice_ind = obj_list.index(d['choice'])
        confusion[targ_ind,choice_ind] += 1

    ## normalized confusion matrix    
    for i in np.arange(len(confusion)):
        normed[i,:,k] = confusion[i,:]/np.sum(confusion[i,:])    

## plot difference in confusion matrix
from matplotlib import cm
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(111)
cax = ax.matshow(normed[:,:,0]-normed[:,:,1],vmin=-0.2,vmax=0.2,cmap=cm.BrBG)
plt.xticks(range(len(normed)), obj_list, fontsize=12,rotation='vertical')
plt.yticks(range(len(normed)), obj_list, fontsize=12)
plt.colorbar(cax,shrink=0.8)
plt.tight_layout()
plt.savefig('./plots/confusion_matrix_close_minus_far.pdf')
plt.close(fig)

# save out to npy 
np.save('./human_confusion.npy',normed)

#### prep jsons for evaluation by passing forward to RSA

Format for json is a dictionary of dictionaries, where each top-level key refers to one of the renders, e.g. "trial_20_cuckoo". For each render, you can look up the similarity with each sketch, referenced with an abbreviated ID taken by trimming the last 12-character string, and appending an underscore, and the trial number. E.g., 'gameID_9903-d6e6a9ff-a878-4bee-b2d5-26e2e239460a_trial_9.npy' ==> '26e2e239460a_9'

In [301]:
import json

def load_json(json_path):
    with open(json_path) as fp:
        data = json.load(fp)  
    return data

## build dictionary to look up the appropriate render ID to use to associate with each sketch
data = load_json(json_path)
    
obj_to_render = dict(zip([i.split('_')[-1] for i in data.keys()], data.keys()))  
render_to_obj = dict(zip(data.keys(),[i.split('_')[-1] for i in data.keys()]))  

In [211]:
def simplify_sketch(path): ## example path: 'gameID_9903-d6e6a9ff-a878-4bee-b2d5-26e2e239460a_trial_9.npy' ==> '26e2e239460a_9'
    path = '_'.join(os.path.splitext(os.path.basename(path))[0].split('_')[1:])
    path = path.split('-')[-1]
    path = path.replace('_trial', '')
    return path

def add_simplified_ids(X):
    ## add renderID and sketchID to dataframe
    renderID = []
    sketchID = []
    for i,d in X.iterrows():
        renderID.append(obj_to_render[d['target']])
        sketchID.append(simplify_sketch(d['fname']))
    X['renderID'] = renderID
    X['sketchID'] = sketchID    
    return X

In [249]:
## add simplified id's
X = add_simplified_ids(X)

## load in normalized confusion matrix
confusion = np.load('./human_confusion.npy')
close_con = confusion[:,:,0]
far_con = confusion[:,:,1]

In [266]:
## get confusion matrix lookup table for sketches from each condition

## close
rows = [i for i in close_con]
close_lookup = dict(zip(obj_list, [dict(zip(obj_list,r)) for r in rows])) 

## far
rows = [i for i in far_con]
far_lookup = dict(zip(obj_list, [dict(zip(obj_list,r)) for r in rows])) 

## list of 3d rendered objects
render_list = data.keys()

In [286]:
## generate big json dictionary of dictionaries
out_json = {}
for i,this_render in enumerate(render_list):
    print i, this_render
    out_json[this_render] = {}
    for i,d in X.iterrows():
        if d['condition'] == 'closer':
            which_dict = close_lookup
        elif d['condition'] == 'further':
            which_dict = far_lookup
        this_sketch = d['sketchID']
        this_target = d['target']
        similarity = which_dict[this_target][render_to_obj[this_render]]
        out_json[this_render][this_sketch] = similarity

0 trial_12_bluejay
1 trial_7_basset
2 trial_2_bullmastiff
3 trial_30_doberman
4 trial_10_beetle
5 trial_11_waiting
6 trial_1_bluesport
7 trial_31_bluesedan
8 trial_4_straight
9 trial_8_knob
10 trial_15_nightingale
11 trial_22_leather
12 trial_24_white
13 trial_28_chihuahua
14 trial_17_redsport
15 trial_29_sparrow
16 trial_21_woven
17 trial_25_goldenretriever
18 trial_32_squat
19 trial_3_redantique
20 trial_9_brown
21 trial_27_sling
22 trial_16_bloodhound
23 trial_23_weimaraner
24 trial_26_crow
25 trial_13_tomtit
26 trial_6_hatchback
27 trial_20_cuckoo
28 trial_18_pug
29 trial_5_inlay
30 trial_19_pigeon
31 trial_14_robin


In [290]:
## output json in the same format as the other similarity jsons
output_path = '../models/refModule/json/similarity-human.json'
with open(output_path, 'wb') as fp:
    json.dump(out_json, fp)

In [302]:
json_path_prefix = '../models/refModule/json/'
json_file = 'strict-similarity-pragmatics-fixedpose-augmented-splitbycontext_conv4_2.json'
x = load_json(os.path.join(json_path_prefix,json_file))

In [309]:
y = pd.read_csv('../models/bdaInput/sketchData_fixedPose_splitbycontext_augmented2_pilot2.csv')

In [310]:
y.head()

,Unnamed: 0,condition,sketchLabel,Target,Distractor1,Distractor2,Distractor3,coarseGrainedSketchInfo
0,0,further,d6a8931c94d5_1,trial_8_knob,trial_17_redsport,trial_26_crow,trial_7_basset,further_knob
1,1,further,d6a8931c94d5_2,trial_17_redsport,trial_26_crow,trial_7_basset,trial_8_knob,further_redsport
2,2,closer,d6a8931c94d5_4,trial_28_chihuahua,trial_18_pug,trial_23_weimaraner,trial_30_doberman,closer_chihuahua
3,3,further,d6a8931c94d5_7,trial_3_redantique,trial_14_robin,trial_27_sling,trial_2_bullmastiff,further_redantique
4,4,further,d6a8931c94d5_8,trial_7_basset,trial_17_redsport,trial_26_crow,trial_8_knob,further_basset
